# 📡 Radio Signal Classifier Integration Demo

本 Notebook 展示如何加载 MS2090A 采样 `.dat` 文件、进行增强、特征转换（S2M）、并使用 ResNet 模型进行调制识别。

In [ ]:
# 安装必要库（如未安装）
!pip install torch torchvision pytorch-lightning numpy

In [ ]:
# 导入核心模块
from resnet_s2m import S2MResNet
from augment import iq_augment
from ms2090a_interface import load_ms2090a_dat_file
import torch, numpy as np
import matplotlib.pyplot as plt

In [ ]:
# 加载 .dat 文件（路径可修改为你的实际数据路径）
dat_path = 'your_signal.dat'  # 替换为你的 MS2090A 采集路径
iq = load_ms2090a_dat_file(dat_path)
iq_aug = iq_augment(iq)

In [ ]:
# S2M 特征转换
s2m = np.outer(iq_aug.real, iq_aug.imag).astype(np.float32)
s2m = (s2m - s2m.mean()) / (s2m.std() + 1e-6)
s2m_tensor = torch.tensor(s2m).unsqueeze(0).unsqueeze(0)  # [1, 1, H, W]

In [ ]:
# 加载模型（假设已训练）
model = S2MResNet(num_classes=11)
# model.load_state_dict(torch.load('model.ckpt', map_location='cpu'))
model.eval()
with torch.no_grad():
    pred = model(s2m_tensor).argmax(dim=1).item()
print(f'识别结果类别编号：{pred}')

In [ ]:
# 可视化 S2M 图像
plt.imshow(s2m, cmap='viridis')
plt.title('Signal-to-Matrix (S2M) Representation')
plt.colorbar()
plt.show()